In [7]:
import numpy as np
import numpy.linalg as lin
from typing import Tuple, List

In [28]:

class TimpaniData:
    def __init__(self, a: float, c_trans: float, alpha: float, b: float, b_3: float):
        self.a = a # Radius of the membrane (m)
        self.c_trans = c_trans # Speed of transversal waves. (m.s-1)
        self.alpha = alpha # (paramètre d'élasticité)
        self.b = b # (paramètre de viscosité)
        self.b_3 = b_3 # (coefficient relié à la force due au rayonnement de la membrane)

        self.w_0 = 0 # value of w outside of the circle.

"""[summary]
Implements « Synthèse sonore de sons de tiimbale, A. Ramdane, 1990
source: http://www.edpsciences.org/10.1051/jphyscol:19902201
Differences in notation :
- w is used as the name for the transversal displacement of the membrane.
- n is used as the number of discrete time points (and not N).
- The time iterator's name is k (and not n).
"""
class TimpaniSolver:
    def __init__(self, data: TimpaniData, dt: float):
        """[summary]
        Args:
            d (TimpaniData): [description]
            steps (Tuple[float]): [description]
            n (int): [description]
        """
        self.data = data
        self.dt = dt
        # dx and dy are set from dt, so that we have r_1**2 + r_2**2 = 1
        self.dx = self.dy = 2 *(self.dt * self.data.c_trans) ** 2
        self.n = int(np.ceil(self.data.a / self.dx))

        print(self.dt)
        print(self.dx)
        print(self.n)

        self.h = self.dx # ? not sure about what that's supposed to be, or the value.

        self.r_1 = self.data.c_trans * self.dt / self.dx
        self.r_2 = self.data.c_trans * self.dt / self.dy

        self.epsilon_1 = lambda i, j, k: np.sqrt(self.n ** 2 - j ** 2)/self.h - i/self.h
        self.epsilon_2 = lambda i, j, k: np.sqrt(self.n ** 2 - j ** 2)/self.h - i/self.h

        # cache of all w(i, j, k) computed for this data
        self.has_w = np.zeros((self.n, self.n, 1), dtype=bool)
        self.w_cache = np.zeros_like(self.has_w)

    def compute_w(self, x: float, y: float, t: float) -> float:
        """[summary]
        (x, y) are normalised by the radius and originate from the centre of the membrane.
        Args:
            x (float): [description]
            y (float): [description]
            t (float): [description]

        Returns:
            float: [description]
        """
        i = int(np.round(((x + 0.5) * a) / self.dx))
        j = int(np.round(((y + 0.5) * a) / self.dy))
        k = int(np.round(t / self.dt))
        return compute_w(i, j, k)
    
    def compute_w(self, i: int, j: int, k: int) -> float:
        """[summary]

        Args:
            i ([type]): [description]
            j ([type]): [description]
            k ([type]): [description]
        """
        if k <= 0:
            # At $t_0$, the displacement is set everywhere on the membrane.
            return self.data.w_0
        # Check whether the value has already been computed.
        if k > self.has_w.shape[2]:
            # extend cache
            self.has_w = np.append(self.has_w, np.zeros(k - self.has_w.shape[2]), axis=-1)
            self.w_cache = np.append(self.w_cache, np.zeros(k - self.has_w.shape[2]), axis=-1)
        #
        if self.has_w[i, j, k]:
            return self.w_cache[i, j, k]
        #
        region = self.__get_point_region((i, j))
        w = None
        if region == 0:
            w = self.d_w_0
        elif region == 1:
            e_1 = self.epsilon(i, j, k)
            e_2 = self.epsilon(i, j, k)
            a = 2 * self.r_1**2 / (1 + e_1 * self.h)
            # pretty sure there is an error in the pdf of the article for this coefficient.
            # for now, I copied it as-is.
            b = - 2 * (self.r_1**2 * (self.h / e_1 + h / e_2) - 1)
            c = 2 * self.r_1**2 / (1 + e_2 * h)
            d = -1
            #
            w = a * self.compute_w(i-1, j, k-1) + \
                b * self.compute_w(i, j, k-1) + \
                c * self.compute_w(i, j-1, k-1) + \
                d * self.compute_w(i, j, k-2)
        else:
            # Re-writing the equation (5) from the article to extract the most up-to-date w (largest k)
            avg_z = 0
            d_x_2_w = lambda i, j, k: self.compute_w(i+1, j, k) - 2 * self.compute_w(i, j, k) + self.compute_w(i-1, j, k)
            d_y_2_w = lambda i, j, k: self.compute_w(i, j+1, k) - 2 * self.compute_w(i, j, k) + self.compute_w(i, j-1, k)
            d_t_w = lambda i, j, k: self.compute_w(i, j, k+1) - self.compute_w(i, j, k)
            d_t_2_w = lambda i, j, k: d_t_w(d_t_w(i, j, k))
            res_w = lambda i, j, k: 2 * self.compute_w(i, j, k+1) + 2 * self.compute_w(i, j, k-1) - self.compute_w(i, j, k-2)
            k_p = k-2 # retard par rapport à la formule de l'article
            w = self.dt**3/self.b_3 * \
                (r_1 ** 2 * d_x_2_z(i, j, k_p) + r_2 ** 2 * d_y_2_z(i, j, k_p) \
                - b/self.dt * d_t_w(i, j, k_p) + self.alpha * avg_z - d_t_2_w(i, j, k_p)) + \
                res_w(i, j, k_p)
        self.w_cache[i, j, k] = w
        self.has_w[i, j, k] = True
        return w


    def __get_point_region(self, p: Tuple[int]) -> int:
        """[summary]
        Get the region of the (i, j) point,
        can be 0: circular rim or outside
            1: neighbourhood of the circular rim on the inside
            2: inside the circle
        Args:
            n ([type]): [description]
            p (Tuple[int]): Point of indices (i, j) in the grid.
        """
        radius = self.n * 0.5
        norm = lin.norm(np.array(p, dtype=np.float64))
        diff = radius - norm
        if norm < radius:
            return 0
        elif norm >= radius - 1:
            return 1
        else:
            return 2 

In [29]:

a = 0.3
c_trans = 500 
alpha = 5 # (paramètre d'élasticité)
b = 0.3 # (paramètre de viscosité)
b_3 = 1 # (coefficient relié à la force due au rayonnement de la membrane)

data = TimpaniData(a, c_trans, alpha, b, b_3)

In [30]:
f_e = 44100
dt = 1/f_e

solver = TimpaniSolver(data, dt)

2.2675736961451248e-05
0.00025709452337246313
1167


In [ ]:
x = - 0.4
y = 0
t = 1

val = solver.compute_w(x, y, t)